In [257]:
import pandas as pd
from trino.dbapi import connect 
from dateutil.relativedelta import relativedelta

### Elig Raw

In [258]:
# # Define the start and end dates
# start_date = '2020-11-01'
# end_date = '2024-03-01'
# 
# # Use f-string for SQL query
# sSQL = f"""
# SELECT 
#     division AS tenantid,
#     division AS company,
#     month,
#     carrier,
#     exchange,
#     tpa,
#     COUNT(*) AS current_member_count,
#     COUNT(DISTINCT employeeid) AS current_employee_count,
#     SUM(total_rate) AS total_rate,
#     SUM(payin_rate) AS paying_rate
# FROM 
#     hive.trinet.elig
# WHERE 
#     status <> 'TERMED'
#     AND month <= DATE('{end_date}')
#     AND month >= DATE('{start_date}')
# GROUP BY 
#     1, 2, 3, 4, 5, 6
# """
# 
# # Use context manager for database connection
# with connect(
#     host='presto.bstis.com',
#     port=8080,
#     user='hadoop',
#     catalog='hive',
# ) as conn:
#     df_elig = pd.read_sql(sSQL, conn)
# 
# # Save DataFrame to a gzip compressed CSV file
# df_elig.to_csv('Elig Data/elig_raw.gz', index=False, compression='gzip')

### Preprocess

In [259]:
from functools import reduce

# Read the gzip compressed CSV file into a DataFrame
df_elig = pd.read_csv('Elig Data/elig_raw.gz', compression='gzip')

# Define the conditions
conditions = [
    "(carrier != 'BCBS MN' | exchange != 'TriNet III')",
    "(carrier != 'Kaiser' | exchange != 'TriNet IV')",
    "(carrier != 'Excellus' | exchange != 'TriNet I')"
]

# Use reduce to apply all the conditions at once
df_elig = df_elig.query(reduce(lambda a, b: a + " & " + b, conditions))
# df_elig

## Set Period

In [260]:
curr_month = (pd.to_datetime(('2024-03-01'))).strftime('%Y-%m-%d')

In [261]:
def calculate_dates(curr_month):
    curr_month = pd.to_datetime(curr_month)
    stop_date_nonkaiser = (curr_month + pd.DateOffset(months=1) - pd.DateOffset(days=1))
    start_date_nonkaiser = (stop_date_nonkaiser - pd.DateOffset(months=12) + pd.DateOffset(days=1))
    stop_date_kaiser = (stop_date_nonkaiser - pd.DateOffset(months=1))
    start_date_kaiser = (start_date_nonkaiser - pd.DateOffset(months=1))
    
    return stop_date_nonkaiser, start_date_nonkaiser, stop_date_kaiser, start_date_kaiser

stop_date_nonkaiser, start_date_nonkaiser, stop_date_kaiser, start_date_kaiser = calculate_dates(curr_month)

start_date_kaiser

Timestamp('2023-03-01 00:00:00')

### Admin Fees

In [262]:
def filter_data(df, carrier, start_date, stop_date, kaiser_dates=None):
    if carrier == 'Kaiser' and kaiser_dates is not None:
        start_date, stop_date = kaiser_dates
        df_filtered = df[df['carrier'] == carrier]
    else:
        df_filtered = df[df['carrier'] != carrier]
    df_filtered = df_filtered[(df_filtered['month'] >= pd.to_datetime(start_date)) & (df_filtered['month'] <= pd.to_datetime(stop_date))]
    return df_filtered

def create_admin_df(df, measure_name, new_column_name):
    df_admin = df[df['Measure Names'] == measure_name]
    df_admin = df_admin.rename(columns={'Measure Values': new_column_name, 'Division': 'tenantid'})
    df_admin = df_admin.groupby(['tenantid','carrier']).agg({new_column_name:'sum'}).reset_index()
    return df_admin

# Read the CSV file into a DataFrame and preprocess the data
admin = pd.read_csv('Elig Data/Admin Fees.csv')
admin['month'] = pd.to_datetime(admin['Month of Eligibility Month'])
admin['carrier'] = admin['Carrier (Tag)']

# Filter the data based on the carrier and the date range
admin_non_kaiser = filter_data(admin, 'Kaiser', start_date_nonkaiser, stop_date_nonkaiser)
admin_kaiser = filter_data(admin, 'Kaiser', start_date_nonkaiser, stop_date_nonkaiser, kaiser_dates=(start_date_kaiser, stop_date_kaiser))

# Concatenate the filtered DataFrames
admin_curr = pd.concat([admin_non_kaiser, admin_kaiser])

# Create the new DataFrames
expense_admin = create_admin_df(admin_curr, 'Expense - Admin (RS Only)', 'admin_expense')
pooling_admin = create_admin_df(admin_curr, 'Expense - Pooling (RS Only)', 'pooling_expense')
other_admin = create_admin_df(admin_curr, 'Expense - Other (RS Only)', 'other_expense')




C:\Users\geoff.joe\AppData\Local\Temp\ipykernel_28280\494838011.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  admin['month'] = pd.to_datetime(admin['Month of Eligibility Month'])


## Elig

Non Kaiser

In [263]:
def calculate_counts(df):
    df['start_member_count'] = df.groupby(['tenantid','company','carrier','exchange'])['current_member_count'].transform('first')
    df['start_employee_count'] = df.groupby(['tenantid','company','carrier','exchange'])['current_employee_count'].transform('first')
    df['stop_member_count'] = df.groupby(['tenantid','company','carrier','exchange'])['current_member_count'].transform('last')
    df['stop_employee_count'] = df.groupby(['tenantid','company','carrier','exchange'])['current_employee_count'].transform('last')
    return df

df_elig_nonkaiser = (df_elig
                     .query("carrier != 'Kaiser'")
                     .assign(month=lambda df: pd.to_datetime(df['month']),
                             curr_month=curr_month,
                             start_date=start_date_nonkaiser,
                             stop_date=stop_date_nonkaiser)
                     .query("@start_date <= month <= @stop_date")
                     .sort_values(by=['tenantid','company','carrier','exchange','month'])
                     .pipe(calculate_counts)
                     .groupby(['tenantid','company','carrier','exchange', 'curr_month', 'start_date', 'stop_date', 'start_member_count', 'stop_member_count', 'start_employee_count', 'stop_employee_count'])
                     .agg({'current_member_count':'sum','current_employee_count':'sum','total_rate':'sum','paying_rate':'sum'})
                     .reset_index()
                     .rename(columns={'current_member_count':'mm_months','current_employee_count':'ee_months'}))

Kaiser

In [264]:
def calculate_counts(df):
    df['start_member_count'] = df.groupby(['tenantid','company','carrier','exchange'])['current_member_count'].transform('first')
    df['start_employee_count'] = df.groupby(['tenantid','company','carrier','exchange'])['current_employee_count'].transform('first')
    df['stop_member_count'] = df.groupby(['tenantid','company','carrier','exchange'])['current_member_count'].transform('last')
    df['stop_employee_count'] = df.groupby(['tenantid','company','carrier','exchange'])['current_employee_count'].transform('last')
    return df

df_elig_kaiser = (df_elig
                  .query("carrier == 'Kaiser'")
                  .assign(month=lambda df: pd.to_datetime(df['month']),
                          curr_month=curr_month,
                          start_date=start_date_nonkaiser,
                          stop_date=stop_date_nonkaiser)
                  .query("@start_date_kaiser <= month <= @stop_date_kaiser")
                  .sort_values(by=['tenantid','company','carrier','exchange','month'])
                  .pipe(calculate_counts)
                  .groupby(['tenantid','company','carrier','exchange', 'curr_month', 'start_date', 'stop_date', 'start_member_count', 'stop_member_count', 'start_employee_count', 'stop_employee_count'])
                  .agg({'current_member_count':'sum','current_employee_count':'sum','total_rate':'sum','paying_rate':'sum'})
                  .reset_index()
                  .rename(columns={'current_member_count':'mm_months','current_employee_count':'ee_months'})
                  )



In [265]:
# Concatenate df_elig_nonkaiser and df_elig_kaiser
df_elig = pd.concat([df_elig_nonkaiser, df_elig_kaiser])

# Merge df_elig with the admin DataFrames
df_elig = pd.merge(df_elig, expense_admin, how='left', on=['tenantid','carrier'])
df_elig = pd.merge(df_elig, pooling_admin, how='left', on=['tenantid','carrier'])
df_elig = pd.merge(df_elig, other_admin, how='left', on=['tenantid','carrier'])

# expense_admin
# Add the new column and save the DataFrame to a CSV file
df_elig.assign(year=lambda df: pd.to_datetime(start_date_nonkaiser).strftime('%Y')).to_csv(f'Elig Data/elig_{curr_month.replace("-", "_")}.csv', index=False)